<a href="https://colab.research.google.com/github/Wonjae98/PUBG_analysis/blob/main/PUBG_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import time

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:
data = pd.read_csv("final_dff.csv")

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3503000 entries, 0 to 3502999
Data columns (total 31 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0.1     int64  
 1   Unnamed: 0       int64  
 2   Id               object 
 3   groupId          object 
 4   matchId          object 
 5   assists          int64  
 6   boosts           int64  
 7   damageDealt      float64
 8   DBNOs            int64  
 9   headshotKills    int64  
 10  heals            int64  
 11  killPlace        int64  
 12  kills            int64  
 13  killStreaks      int64  
 14  matchDuration    int64  
 15  matchType        object 
 16  maxPlace         int64  
 17  revives          int64  
 18  rideDistance     float64
 19  roadKills        int64  
 20  swimDistance     float64
 21  teamKills        int64  
 22  vehicleDestroys  int64  
 23  walkDistance     float64
 24  weaponsAcquired  int64  
 25  winPlacePerc     float64
 26  teamWork         int64  
 27  headshotRati

In [ ]:
df = data.drop(columns = ['Unnamed: 0','Unnamed: 0.1', 'Id', 'groupId', 'matchId'])

In [ ]:
# 데이터 필터링
df_duo = df[df['matchType'] == 'duo']

# 특성과 타겟 변수 분리
X = df_duo.drop(columns=['winPlacePerc', 'matchType'])
y = df_duo['winPlacePerc']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 라쏘 회귀 모델 설정 및 하이퍼파라미터 그리드 서치
lasso = Lasso(random_state=42)
parameters = {
    'alpha': [0.01, 0.1, 1, 10, 100]  # 라쏘 회귀의 규제 강도 하이퍼파라미터
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(lasso, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_lasso = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_duo['Predicted_winPlacePerc'] = best_lasso.predict(X)

# 성능 지표 계산
y_pred = best_lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_duo_lasso = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 제외된 변수 확인
excluded_variables = X.columns[(best_lasso.coef_ == 0)].tolist()

# 결과 출력
results_duo_lasso_df = pd.DataFrame([results_duo_lasso])
results_duo_lasso_df

Fitting 3 folds for each of 5 candidates, totalling 15 fits


<ipython-input-10-92142e9c7709>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duo['Predicted_winPlacePerc'] = best_lasso.predict(X)


,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.01297,0.085242,0.113886,0.849288,0.849272,{'alpha': 0.01},19.648429


In [ ]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import numpy as np

# 리지 회귀 모델 설정 및 하이퍼파라미터 그리드 서치
ridge = Ridge(random_state=42)
parameters = {
    'alpha': [0.01, 0.1, 1, 10, 100]  # 리지 회귀의 규제 강도 하이퍼파라미터
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(ridge, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_ridge = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_duo['Predicted_winPlacePerc'] = best_ridge.predict(X)

# 성능 지표 계산
y_pred = best_ridge.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_duo_ridge = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 결과 출력
results_duo_ridge_df = pd.DataFrame([results_duo_ridge])
results_duo_ridge_df

Fitting 3 folds for each of 5 candidates, totalling 15 fits


<ipython-input-11-7aeb1a2588e5>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duo['Predicted_winPlacePerc'] = best_ridge.predict(X)


,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.011573,0.079286,0.107577,0.865524,0.86551,{'alpha': 0.1},8.25386


In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
parameters = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(xg_reg, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_xgb = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_duo['Predicted_winPlacePerc'] = best_xgb.predict(X)

# 성능 지표 계산
y_pred = best_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_duo_xgb = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 결과 출력
results_duo_xgb_df = pd.DataFrame([results_duo_xgb])
print(results_duo_xgb_df)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


<ipython-input-12-f0909b347654>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duo['Predicted_winPlacePerc'] = best_xgb.predict(X)


        MSE      MAE      RMSE  R-squared  Adjusted R-squared  \
0  0.004501  0.04882  0.067092   0.947694            0.947689   

                                         Best Params  Execution Time (s)  
0  {'colsample_bytree': 0.8, 'learning_rate': 0.2...         6214.540134  


In [ ]:
# Random Forest 회귀 모델 설정 및 하이퍼파라미터 그리드 서치
rf = RandomForestRegressor(random_state=42)
parameters = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(rf, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_duo['Predicted_winPlacePerc'] = best_rf.predict(X)

# 성능 지표 계산
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_duo_rf = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 결과 출력
results_duo_rf_df = pd.DataFrame([results_duo_rf])
print(results_duo_rf_df)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


<ipython-input-13-24fd4a78b577>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duo['Predicted_winPlacePerc'] = best_rf.predict(X)


        MSE       MAE     RMSE  R-squared  Adjusted R-squared  \
0  0.004938  0.051292  0.07027   0.942622            0.942616   

                                         Best Params  Execution Time (s)  
0  {'max_depth': 20, 'max_features': 'auto', 'min...        16894.575011  


In [ ]:
## solo

In [ ]:
# 데이터 필터링
df_solo = df[df['matchType'] == 'solo']

# 특성과 타겟 변수 분리
X = df_solo.drop(columns=['winPlacePerc', 'matchType'])
y = df_solo['winPlacePerc']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 라쏘 회귀 모델 설정 및 하이퍼파라미터 그리드 서치
lasso = Lasso(random_state=42)
parameters = {
    'alpha': [0.01, 0.1, 1, 10, 100]  # 라쏘 회귀의 규제 강도 하이퍼파라미터
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(lasso, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_lasso = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_solo['Predicted_winPlacePerc'] = best_lasso.predict(X)

# 성능 지표 계산
y_pred = best_lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_solo_lasso = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 제외된 변수 확인
excluded_variables = X.columns[(best_lasso.coef_ == 0)].tolist()

# 결과 출력
results_solo_lasso_df = pd.DataFrame([results_solo_lasso])
results_solo_lasso_df

Fitting 3 folds for each of 5 candidates, totalling 15 fits


<ipython-input-15-a15a03a98290>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_solo['Predicted_winPlacePerc'] = best_lasso.predict(X)


,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.010832,0.079101,0.104078,0.872638,0.872615,{'alpha': 0.01},11.324552


In [ ]:
# 리지 회귀 모델 설정 및 하이퍼파라미터 그리드 서치
ridge = Ridge(random_state=42)
parameters = {
    'alpha': [0.01, 0.1, 1, 10, 100]  # 리지 회귀의 규제 강도 하이퍼파라미터
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(ridge, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_ridge = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_solo['Predicted_winPlacePerc'] = best_ridge.predict(X)

# 성능 지표 계산
y_pred = best_ridge.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_solo_ridge = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 결과 출력
results_solo_ridge_df = pd.DataFrame([results_solo_ridge])
results_solo_ridge_df

Fitting 3 folds for each of 5 candidates, totalling 15 fits


<ipython-input-16-a34020a95236>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_solo['Predicted_winPlacePerc'] = best_ridge.predict(X)


,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.008901,0.068443,0.094345,0.895346,0.895328,{'alpha': 0.1},1.581356


In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
parameters = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(xg_reg, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_xgb = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_solo['Predicted_winPlacePerc'] = best_xgb.predict(X)

# 성능 지표 계산
y_pred = best_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_solo_xgb = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 결과 출력
results_solo_xgb_df = pd.DataFrame([results_solo_xgb])
results_solo_xgb_df

Fitting 3 folds for each of 72 candidates, totalling 216 fits


<ipython-input-17-57a1029aae1d>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_solo['Predicted_winPlacePerc'] = best_xgb.predict(X)


,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.003109,0.039795,0.055757,0.963447,0.963441,"{'colsample_bytree': 0.8, 'learning_rate': 0.1...",4298.071512


In [ ]:
# Random Forest 회귀 모델 설정 및 하이퍼파라미터 그리드 서치
rf = RandomForestRegressor(random_state=42)
parameters = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(rf, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_solo['Predicted_winPlacePerc'] = best_rf.predict(X)

# 성능 지표 계산
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_solo_rf = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 결과 출력
results_solo_rf_df = pd.DataFrame([results_solo_rf])
results_solo_rf_df

Fitting 3 folds for each of 72 candidates, totalling 216 fits


<ipython-input-18-0c4504c150a7>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_solo['Predicted_winPlacePerc'] = best_rf.predict(X)


,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.003449,0.04189,0.058726,0.959451,0.959444,"{'max_depth': 20, 'max_features': 'auto', 'min...",7576.780525


In [ ]:
# squad

In [ ]:
# 데이터 필터링
df_squad = df[df['matchType'] == 'squad']

# 특성과 타겟 변수 분리
X = df_squad.drop(columns=['winPlacePerc', 'matchType'])
y = df_squad['winPlacePerc']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 라쏘 회귀 모델 설정 및 하이퍼파라미터 그리드 서치
lasso = Lasso(random_state=42)
parameters = {
    'alpha': [0.01, 0.1, 1, 10, 100]  # 라쏘 회귀의 규제 강도 하이퍼파라미터
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(lasso, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_lasso = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_squad['Predicted_winPlacePerc'] = best_lasso.predict(X)

# 성능 지표 계산
y_pred = best_lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_squad_lasso = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 제외된 변수 확인
excluded_variables = X.columns[(best_lasso.coef_ == 0)].tolist()

# 결과 출력
results_squad_lasso_df = pd.DataFrame([results_squad_lasso])
results_squad_lasso_df

Fitting 3 folds for each of 5 candidates, totalling 15 fits


<ipython-input-20-7ef4c60a77a7>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_squad['Predicted_winPlacePerc'] = best_lasso.predict(X)


,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.019541,0.104747,0.139788,0.777501,0.777485,{'alpha': 0.01},23.274245


In [ ]:
# 리지 회귀 모델 설정 및 하이퍼파라미터 그리드 서치
ridge = Ridge(random_state=42)
parameters = {
    'alpha': [0.01, 0.1, 1, 10, 100]  # 리지 회귀의 규제 강도 하이퍼파라미터
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(ridge, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_ridge = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_squad['Predicted_winPlacePerc'] = best_ridge.predict(X)

# 성능 지표 계산
y_pred = best_ridge.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_squad_ridge = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 결과 출력
results_squad_ridge_df = pd.DataFrame([results_squad_ridge])
results_squad_ridge_df

Fitting 3 folds for each of 5 candidates, totalling 15 fits


<ipython-input-21-71aa891ea6bf>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_squad['Predicted_winPlacePerc'] = best_ridge.predict(X)


,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.01807,0.100491,0.134426,0.794241,0.794226,{'alpha': 0.01},3.545799


In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
parameters = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(xg_reg, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_xgb = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_squad['Predicted_winPlacePerc'] = best_xgb.predict(X)

# 성능 지표 계산
y_pred = best_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_squad_xgb = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 결과 출력
results_squad_xgb_df = pd.DataFrame([results_squad_xgb])
results_squad_xgb_df

Fitting 3 folds for each of 72 candidates, totalling 216 fits


<ipython-input-22-38050a397c61>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_squad['Predicted_winPlacePerc'] = best_xgb.predict(X)


,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.007904,0.066383,0.088903,0.910004,0.909997,"{'colsample_bytree': 1.0, 'learning_rate': 0.2...",8463.958462


In [ ]:
# Random Forest 회귀 모델 설정 및 하이퍼파라미터 그리드 서치
rf = RandomForestRegressor(random_state=42)
parameters = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# 코드 실행 시간 측정 시작
start_time = time.time()

grid_search = GridSearchCV(rf, parameters, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

# 전체 데이터에 대해 예측 수행
df_squad['Predicted_winPlacePerc'] = best_rf.predict(X)

# 성능 지표 계산
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
adj_r2 = 1 - (1-r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# 코드 실행 시간 측정 종료
end_time = time.time()
execution_time = end_time - start_time

results_squad_rf = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'R-squared': r2,
    'Adjusted R-squared': adj_r2,
    'Best Params': grid_search.best_params_,
    'Execution Time (s)': execution_time
}

# 결과 출력
results_squad_rf_df = pd.DataFrame([results_squad_rf])
results_squad_rf_df

Fitting 3 folds for each of 72 candidates, totalling 216 fits


<ipython-input-23-b9d379fff0fd>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_squad['Predicted_winPlacePerc'] = best_rf.predict(X)


,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.008506,0.069178,0.092227,0.903148,0.903141,"{'max_depth': 20, 'max_features': 'auto', 'min...",29218.651021


In [ ]:
pd.set_option('display.max_colwidth', None)
results_squad_lasso_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.019541,0.104747,0.139788,0.777501,0.777485,{'alpha': 0.01},23.274245


In [ ]:
results_squad_ridge_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.01807,0.100491,0.134426,0.794241,0.794226,{'alpha': 0.01},3.545799


In [ ]:
results_squad_xgb_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.007904,0.066383,0.088903,0.910004,0.909997,"{'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 200, 'subsample': 1.0}",8463.958462


In [ ]:
results_squad_rf_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.008506,0.069178,0.092227,0.903148,0.903141,"{'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}",29218.651021


In [ ]:
results_solo_lasso_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.010832,0.079101,0.104078,0.872638,0.872615,{'alpha': 0.01},11.324552


In [ ]:
results_solo_ridge_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.008901,0.068443,0.094345,0.895346,0.895328,{'alpha': 0.1},1.581356


In [ ]:
results_solo_xgb_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.003109,0.039795,0.055757,0.963447,0.963441,"{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'subsample': 1.0}",4298.071512


In [ ]:
results_solo_rf_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.003449,0.04189,0.058726,0.959451,0.959444,"{'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}",7576.780525


In [ ]:
results_duo_lasso_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.01297,0.085242,0.113886,0.849288,0.849272,{'alpha': 0.01},19.648429


In [ ]:
results_duo_ridge_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.011573,0.079286,0.107577,0.865524,0.86551,{'alpha': 0.1},8.25386


In [ ]:
results_duo_xgb_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.004501,0.04882,0.067092,0.947694,0.947689,"{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 200, 'subsample': 1.0}",6214.540134


In [ ]:
results_duo_rf_df

,MSE,MAE,RMSE,R-squared,Adjusted R-squared,Best Params,Execution Time (s)
0,0.004938,0.051292,0.07027,0.942622,0.942616,"{'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}",16894.575011
